In [2]:
#dataset link: https://drive.google.com/drive/folders/1n67sVTTzye4jtLfk8n-sa2fH2gTx5Ywt?usp=share_link

In [3]:
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
DATA_DIR = "./Dataset Korosi/"

In [4]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
    )

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    
)

train_generator = training_datagen.flow_from_directory(DATA_DIR, batch_size=32,class_mode='categorical',target_size=(150,150))   
val_generator = train_generator = training_datagen.flow_from_directory(DATA_DIR, subset='validation', batch_size=32,class_mode='categorical',target_size=(150,150))   

Found 304 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [5]:
resnet = tf.keras.applications.ResNet50(
    input_shape=(150,150,3),
    weights='imagenet',
    include_top=False,
)

In [6]:
model_resnet = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer( input_shape=(150,150,3)),
    resnet,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])


In [7]:
import matplotlib.pyplot as plt

In [8]:
f = open("resnet_50.csv", "w")
f.write("epochs,loss,val_loss,acc,val_acc")
f.close()


In [9]:
Wsave = model.get_weights()
for times in range(10):
    epo = (times+1)*10
    model.set_weights(Wsave)
    model_resnet.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])
    history = model_resnet.fit(train_generator,validation_data=val_generator,epochs=epo, verbose = 0)

    f = open("resnet_50.csv", "a")
    f.write("\n"+str(epo)+","+
        str(history.history['loss'][-1])+","+
        str(history.history['val_loss'][-1])+","+
        str(history.history['acc'][-1])+","+
        str(history.history['val_acc'][-1])
    )
    f.close()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Model')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./resnet50/loss_'+str(epo)+'.png')
    plt.cla()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./resnet50/acc_'+str(epo)+'.png')
    
    plt.clf()

Epoch 1/100
2/2 [==============================] - 18s 6s/step - loss: 17.2084 - acc: 0.4167 - val_loss: 61.9699 - val_acc: 0.3167
Epoch 2/100
2/2 [==============================] - 6s 4s/step - loss: 7.8664 - acc: 0.7000 - val_loss: 1131.4159 - val_acc: 0.3167
Epoch 3/100
2/2 [==============================] - 6s 4s/step - loss: 0.3653 - acc: 0.9167 - val_loss: 2259.8469 - val_acc: 0.3167
Epoch 4/100
2/2 [==============================] - 6s 4s/step - loss: 1.8917 - acc: 0.8833 - val_loss: 94589.9531 - val_acc: 0.3167
Epoch 5/100
2/2 [==============================] - 6s 4s/step - loss: 0.9292 - acc: 0.9000 - val_loss: 253054.5000 - val_acc: 0.3167
Epoch 6/100
2/2 [==============================] - 6s 4s/step - loss: 0.0098 - acc: 1.0000 - val_loss: 458621.9062 - val_acc: 0.3167
Epoch 7/100
2/2 [==============================] - 6s 4s/step - loss: 0.5787 - acc: 0.9167 - val_loss: 1553600.6250 - val_acc: 0.3167
Epoch 8/100
2/2 [==============================] - 6s 3s/step - loss: 0.091

<Figure size 432x288 with 0 Axes>